In [15]:
%pylab
from sys import argv
import h5py
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import os.path as osp

m83_data = np.loadtxt("m83_clusters.dat")
m83_masses = 10**m83_data[:,8]
m83_masses_max = 10**m83_data[:,11]
m83_masses_min = 10**m83_data[:,14]
m83_RGC = 10**m83_data[:,5]
m83_class = m83_data[:,6]
m83_age = 10**m83_data[:,7]
#(m83_masses_max-m83_masses_min < m83_masses) * 
cut = (m83_age > 1e7) * (m83_RGC >450) * (m83_age < 2e8)
m83_masses = m83_masses[cut]
m83_masses_max = m83_masses_max[cut]
m83_masses_min = m83_masses_min[cut]
plt.loglog(np.sort(m83_masses_max), np.arange(len(m83_masses_max))[::-1]+1)
plt.loglog(np.sort(m83_masses_min), np.arange(len(m83_masses_min))[::-1]+1)
plt.loglog(np.sort(m83_masses), np.arange(len(m83_masses))[::-1]+1)

#m51_data = np.loadtxt("m51_legus.tab")
#m51_mass = m51_data[:,20]
#m51_class = m51_data[:,-2]
#m51_mass = m51_mass[m51_class == 1]
#plt.loglog(np.sort(m51_mass), np.arange(len(m51_mass))[::-1]+1)
#plt.show()

#m31_data = np.loadtxt("m31.dat")
#m31_age = 10**m31_data[:,0]
#m31_m = 10**m31_data[:,1]
#m31_m = m31_m[(m31_age > 1e7)*(m31_age < 2e8)]

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [13]:
#data = np.loadtxt("ClusterPopulation_1_reduced.dat")
a = np.loadtxt("snapshot_times.txt")[1:,1]
t = np.loadtxt("snapshot_times.txt")[1:,3]
n = np.loadtxt("snapshot_times.txt")[1:,0]
dt = np.loadtxt("snapshot_times.txt")[1:,-1]
#sfrdata = np.loadtxt("m12i_res7100_mhdcv_sfr.dat")[1:-10]
#sfrdict=  dict(zip(sfrdata[:,0], sfrdata[:,1]))
#mdict = dict(zip(sfrdata[:,0], sfrdata[:,3]))
#F = h5py.File("ClusterPopulation_1.hdf5",'r')
#redshifts = 1/a - 1
redshifts = np.loadtxt("snapshot_times.txt")[:,2]
times = np.loadtxt("snapshot_times.txt")[:,3]

In [4]:
from glob import glob
t = []
rh = []
m = []
z = []
for f in sorted(glob("../cloud_catalogue/metal_diffusion/runs/m12i_res7100/ClusterPopulation_*")):
    with h5py.File(f,'r') as F:
        if not [k for k in F.keys()]: continue
        m.append(np.array(F["Mass"]))
        idx = m[-1] > 1e4
        m[-1] = m[-1][idx]
        t.append(np.array(F["FormationTime"])[idx])
        z.append(np.array(F["Metallicity"])[idx])
        rh.append(np.array(F["HalfMassRadius"])[idx])
m = np.concatenate(m)
t = np.concatenate(t)
z = np.log10(np.concatenate(z)) - np.log10(0.02)
rh = np.concatenate(rh)

In [5]:
from glob import glob

#fig_massfunc,ax = plt.subplots(figsize=(4,4))
def GetClusterProperties(path):
    t = []
    rh = []
    m = []
    z = []
    for f in sorted(glob(path+"/ClusterPopulation_*")):
        with h5py.File(f,'r') as F:
            if not [k for k in F.keys()]: continue
            m.append(np.array(F["Mass"]))
            idx = m[-1] > 1e4
            m[-1] = m[-1][idx]
            t.append(np.array(F["FormationTime"])[idx])
            z.append(np.array(F["Metallicity"])[idx])
            rh.append(np.array(F["HalfMassRadius"])[idx])
    m = np.concatenate(m)
    t = np.concatenate(t)
    z = np.log10(np.concatenate(z)) - np.log10(0.02)
    rh = np.concatenate(rh)
    return m, t, z, rh

def PlotAgeMassMetallicity(path):
    plt.clf()
    plt.close()
    print(path)
    name = path.split("runs/")[1].split("/")[0]
    m, t, z, rh = GetClusterProperties(path)

    fig, axes = plt.subplots(1,3, figsize=(8,3))
    #for f in "ClusterPopulation_1.hdf5",:
    #    F = h5py.File(f,'r')
    #    filt = np.array(F["Mass"]) > 1e4
    Z = z#np.log10(F["Metallicity"])[filt] - np.log10(0.02)
    M = m#np.array(F["Mass"])[filt]
    tf = 13.79874688 - t
    axes[0].scatter(tf, Z,s=.1,color='black', marker='o',alpha=0.5,facecolor='black')
    axes[0].set_xlabel("Age (Gyr)")
    axes[0].set_ylabel("Metallicity")
    axes[0].set_ylim(-4,1)
    ax2 = axes[0].twiny()
    ax2.set_xlim(axes[0].get_xlim())
    ax2.set_xticks(times.max() - np.sort(np.interp(np.arange(0,5)[::-1], np.sort(redshifts), times[redshifts.argsort()])))
    ax2.set_xticklabels([str(a) for a in np.arange(0,5)][::-1])
    ax2.set_xlabel(r"$z$")
    ax2.minorticks_off()
    ax2.set_xlim(0,13.8)
    axes[0].set_xlim(0,13.8)
    axes[2].scatter(tf, M,s=.1,color='black', marker='o',alpha=0.5,facecolor='black')
    axes[2].set_xlabel("Age (Gyr)")
    axes[2].set_ylabel("Mass ($M_\odot$)")
    axes[2].set_ylim(1e4,1e7)
    axes[2].set_yscale('log')
    axes[2].set_xlim(0,13.8)
    ax3 = axes[2].twiny()
    ax3.set_xlim(axes[2].get_xlim())
    ax3.set_xticks(times.max() - np.sort(np.interp(np.arange(0,5)[::-1], np.sort(redshifts), times[redshifts.argsort()])))
    ax3.set_xticklabels([str(a) for a in np.arange(0,5)][::-1])
    ax3.set_xlabel(r"$z$")
    ax3.set_xlim(axes[2].get_xlim())
    axes[1].scatter(M, Z, s=.1, color='black', marker='o',alpha=0.5,facecolor='black')
    axes[1].set_xlabel("Mass ($M_\odot$)")
    axes[1].set_xscale('log')
    axes[1].set_xlim(1e4,1e7)
    axes[1].set_xticks([1e4,1e5,1e6,1e7])
    axes[1].set_ylabel("Metallicity")
    axes[1].set_ylim(-4,1)
    #    plt.xscale('log')
    axes[1].set_title(name.replace("_","\_"))
    plt.tight_layout()
        
    plt.savefig("Mass_Age_Metallicity_%s.png"%name, bbox_inches='tight', dpi=400)
    
for k in glob("../cloud_catalogue/metal_diffusion/runs/m12?_res????"):
    try:
        PlotAgeMassMetallicity(k)
    except:
        print("no dice for %s"%k)
        continue

../cloud_catalogue/metal_diffusion/runs/m12r_res7100


/Users/mgrudic/intelpython3/lib/python3.7/site-packages/ipykernel_launcher.py:71: UserWarning: AutoMinorLocator does not work with logarithmic scale
/Users/mgrudic/intelpython3/lib/python3.7/site-packages/matplotlib/ticker.py:2707: RuntimeWarning: invalid value encountered in log10
  majorstep_no_exponent = 10 ** (np.log10(majorstep) % 1)
/Users/mgrudic/intelpython3/lib/python3.7/site-packages/ipykernel_launcher.py:73: UserWarning: AutoMinorLocator does not work with logarithmic scale
/Users/mgrudic/intelpython3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.


../cloud_catalogue/metal_diffusion/runs/m12f_res7100
no dice for ../cloud_catalogue/metal_diffusion/runs/m12f_res7100
../cloud_catalogue/metal_diffusion/runs/m12m_res7100
no dice for ../cloud_catalogue/metal_diffusion/runs/m12m_res7100
../cloud_catalogue/metal_diffusion/runs/m12i_res7100
../cloud_catalogue/metal_diffusion/runs/m12z_res4200
no dice for ../cloud_catalogue/metal_diffusion/runs/m12z_res4200
../cloud_catalogue/metal_diffusion/runs/m12c_res7100
../cloud_catalogue/metal_diffusion/runs/m12w_res7100
../cloud_catalogue/metal_diffusion/runs/m12b_res7100


Plot mass functions

In [36]:
np.interp(6.,times, redshifts)

0.997108036457213

In [3]:
fig, ax = plt.subplots(figsize=(4,4))

from palettable.colorbrewer.qualitative import Set2_7
ax.set_prop_cycle('color', Set2_7.mpl_colors)


for path in glob("../cloud_catalogue/metal_diffusion/runs/m12?_res????"):
    name = path.split("runs/")[1].split("/")[0]
    try:
        m, t, z, rh = GetClusterProperties(path)
    except:
        continue
    ax.plot(np.sort(m),np.arange(len(m))[::-1]+1, label=name.replace("_","\_"))
ax.set_xscale('log')
ax.set_xlim(1e4,1e7)
ax.set_ylim(1,1e4)
ax.set_yscale('log')
ax.legend()
plt.savefig("MassFunctions.pdf",bbox_inches='tight')

findfont: Font family ['serif'] not found. Falling back to DejaVu Sans.


In [17]:
plt.clf()
plt.close()
fig, ax = plt.subplots(figsize=(4,4))
from palettable.colorbrewer.qualitative import Dark2_7
ax.set_prop_cycle('color', Dark2_7.mpl_colors)

from numba import njit
@njit
def smooth_timeseries(series, tau):
    smoothed = np.empty_like(series)
    smoothed[0] = series[0]
    value = smoothed[0]
    for i in range(1,len(series)):
        value = (1 - 1./tau) * value + 1./tau * series[i]
        smoothed[i] = value
    return smoothed
    

for path in glob("../cloud_catalogue/metal_diffusion/runs/m12?_res????"):
    name = path.split("runs/")[1].split("/")[0]
    try:
        data = np.loadtxt(path+"/SFR.txt")
    except:
        continue
    t = data[:,3]
    SFR = data[:,6]
    mstar = data[:,-1]
    print(name, t[mstar.cumsum() > mstar.sum()/2][0])
    
    ax.plot(t, smooth_timeseries(SFR,10.), label=name.replace("_","\_"),lw=1)
ax.set_ylabel("SFR ($M_\odot\,\mathrm{yr}^{-1}$)")
ax.set_xlabel("Time (Gyr)")
ax.set_yscale('log')
ax.set_ylim(0.1,100)
ax.legend(labelspacing=0)
plt.savefig("SFR.pdf",bbox_inches='tight')             
#    print(data[-1])

m12r_res7100 9.865528
m12f_res7100 9.94001741
m12m_res7100 6.69390806
m12i_res7100 8.8383935
m12c_res7100 9.81323434
m12w_res7100 8.12883828
m12b_res7100 9.6351146


Plot SFH's

In [12]:
from palettable.colorbrewer.qualitative import Set2_4
agebins = t.max() * np.linspace(0.0,1,200)

F = h5py.File("ClusterPopulation_1.hdf5",'r')
tf = np.array(F["FormationTime"])
print(t.size, dt.size)
dt_tf = np.interp(tf, t, dt)

plt.clf()
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111)
ax2 = ax.twiny()
ax.set_prop_cycle('color', Set2_4.mpl_colors)

for masscut in 0, 1e4, 1e5, 1e6:
    print(masscut)
    filt = np.array(F["Mass"]) > masscut
    #agebins = np.concatenate([[0],t])
    Z = np.log10(F["Metallicity"])[filt] - np.log10(0.02)
    M = np.array(F["Mass"])[filt]
    tf = np.array(F["FormationTime"])[filt]
    
    mdot_cluster = np.histogram(tf, agebins, weights=M)[0] / np.diff(1e9*agebins)

    sfr_snap = sfrdata[:,3] / (dt*1e6)
    mdot_star = np.histogram(t, agebins, weights=sfrdata[:,3])[0] / np.diff(agebins) / 1e9
    CFE = mdot_cluster / mdot_star
    
    if masscut == 1e6:
        ax.fill_between(agebins[1:], CFE, np.zeros_like(CFE), alpha=1, label=("All bound clusters" if masscut==0 else r"$M_\mathrm{cl}>10^{%d}M_\odot$"%int(np.log10(masscut)+0.5)), color='red')
        ax.plot(agebins[1:], CFE,color='red',lw=0.3)
    else:
        ax.fill_between(agebins[1:], CFE, np.zeros_like(CFE), alpha=1, label=("All bound clusters" if masscut==0 else r"$M_\mathrm{cl}>10^{%d}M_\odot$"%int(np.log10(masscut)+0.5)))
        ax.plot(agebins[1:], CFE,color='black',lw=0.3)
ax.set_ylim(1e-4,3)
ax.legend(labelspacing=0)
ax.set_yscale('log')
ax.set_xlabel("Cosmic Time [Gyr]")
ax.set_ylabel("Cluster Formation Efficiency $\Gamma$")
ax.set_xlim(0,13.8)
ax2.set_xlim(ax.get_xlim())
ax2.set_xticks(np.sort(np.interp(np.arange(0,6), np.sort(redshifts), times[redshifts.argsort()])))
ax2.set_xticklabels([str(a) for a in np.arange(0,6)[::-1]])
ax2.set_xlabel(r"$z$")
ax2.minorticks_off()
ax2.set_xlim(0,13.8)
plt.savefig("Time_vs_Gamma.pdf", bbox_inches='tight')

4696 600


ValueError: fp and xp are not of the same length.

In [155]:
np.interp(5, np.sort(redshifts), times[redshifts.argsort()])

1.2089659

In [2]:
from palettable.colorbrewer.qualitative import Set2_4
from scipy.stats import binned_statistic

agebins = t.max() * np.linspace(0.0,1,600)

F = h5py.File("ClusterPopulation_1.hdf5",'r')
tf = np.array(F["FormationTime"])
dt_tf = np.interp(tf, t, dt)

plt.clf()
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111)
ax2 = ax.twiny()
#ax.set_prop_cycle('color', Set2_4.mpl_colors)

#for masscut in 0, 1e4, 1e5, 1e6:
#    print(masscut)
filt = np.array(F["Mass"]) > 0
    #agebins = np.concatenate([[0],t])
Z = np.log10(F["Metallicity"])[filt] - np.log10(0.02)
M = np.array(F["Mass"])[filt]
tf = np.array(F["FormationTime"])[filt]

massmed = lambda x: np.interp(x.sum()/2, np.sort(x).cumsum(), np.sort(x))

mmax_cluster = binned_statistic(tf, M, statistic='max', bins=agebins)[0]
#mmed_cluster = binned_statistic(tf, M, statistic=massmed, bins=agebins)[0]
  
ax.plot(agebins[1:], mmax_cluster, color='black')
#ax.plot(agebins[1:], mmed_cluster)
#ax.set_ylim(1e-4,3)
ax.set_xlim(0,13.8)

ax.legend(labelspacing=0)
ax.set_yscale('log')
ax.set_xlabel("Cosmic Time [Gyr]")
ax.set_ylabel(r"Maximum Cluster Mass [$M_\odot$]")
ax2.set_xlim(ax.get_xlim())
ax2.set_xticks(np.sort(np.interp(np.arange(0,6), np.sort(redshifts), times[redshifts.argsort()])))
ax2.set_xticklabels([str(a) for a in np.arange(0,6)[::-1]])
ax2.set_xlabel(r"$z$")
ax2.set_xlim(0,13.8)
ax2.minorticks_off()
plt.savefig("Time_vs_Mcluster.pdf", bbox_inches='tight')

NameError: name 't' is not defined

In [21]:
from palettable.colorbrewer.qualitative import Set2_4
from scipy.stats import binned_statistic

agebins = t.max() * np.linspace(0.0,1,int(t.max()/0.1+1))

F = h5py.File("ClusterPopulation_1_classic.hdf5",'r')
tf = np.array(F["FormationTime"])
dt_tf = np.interp(tf, t, dt)

plt.clf()
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111)

tf = np.array(F["FormationTime"])
M = np.array(F["Mass"])
for i in range(len(agebins)-1):    
    filt = (tf > agebins[i])*(tf<=agebins[i+1])*(M>1e3)
    mbin = M[filt]
    ax.plot(np.sort(mbin),(np.arange(len(mbin))[::-1]+1), color='black',alpha=0.05, label=("$100\mathrm{Myr}$ age bins" if i==0 else None))
    #agebins = np.concatenate([[0],t])
#Z = np.log10(F["Metallicity"])[filt] - np.log10(0.02)
ax.plot(np.sort(M[M>1e3]), (np.arange(len(M[M>1e3]))[::-1]+1),label="Total")

# plot m83 CDF from Adamo 2015
# m83_masses = 10**np.loadtxt("m83_mass_cdf_adamo2015.csv",delimiter=',')[:,0]
# m83_cdf = np.loadtxt("m83_mass_cdf_adamo2015.csv",delimiter=',')[:,1]
# m83_cdf /= m83_cdf.min()
# ax.plot(np.sort(m83_masses), m83_cdf[m83_masses.argsort()],color='red', label="M83 (Adamo+15)")

#  M51 CDF from Messa 2018
# ax.plot(np.sort(m51_mass), np.arange(len(m51_mass))[::-1]+1, label="M51 (Messa+18)")

# M31 CDF from Johnson 2016
# ax.plot(np.sort(m31_m), np.arange(len(m31_m))[::-1]+1, label="M31 (Johnson+18)")

ax.text(1.1e6,2000,"$\propto M_\mathrm{cl}^{-2}$", rotation=-60)
ax.text(6e5,350,"$\propto M_\mathrm{cl}^{-1}$", rotation=-37)
ax.set_xlim(1e3,1e7)
ax.set_ylim(1,1e5)
ax.plot([5e5,5e6],[3000,300],color='black',ls='dotted')
ax.plot([5e5,5e6],[30000,300],color='black',ls='dotted')
ax.legend(labelspacing=0,frameon=True,loc=2)
ax.set_ylabel(r"$N\left(>M_\mathrm{cl}\right)$")
ax.set_xlabel(r"$M_\mathrm{cl}$ [$M_\odot$]")
ax.set_yscale('log')
ax.set_xscale('log')
#ax.set_xlabel("Cosmic Time [Gyr]")
#ax.set_ylabel(r"Maximum Cluster Mass [$M_\odot$]")
#plt.savefig("Time_vs_MassFunction.pdf", bbox_inches='tight')
plt.savefig("Time_vs_MassFunction_classic.pdf", bbox_inches='tight')

In [35]:
plt.loglog(np.sort(m83_masses), m83_cdf[m83_masses.argsort()])
plt.show()

/home/mike/.conda/envs/idp/lib/python3.6/tkinter/__init__.py:745: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)


In [ ]:
Myoung = M[tf>13.7]
plt.plot(M[M.argsort()], M[M.argsort()].cumsum())
plt.xscale('log')

In [ ]:
corr =np.average(1/dt[1:]) * dt

In [ ]:
M.min()

In [ ]:
len(M)/1e6

In [ ]:
t